<a href="https://colab.research.google.com/github/emolinaperez/econometrics_mek/blob/main/Week%202/Python/Lab2_Econometrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Lab 2: Econometrics**

Import Required Libraries

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import ttest_ind
import os
from scipy.stats import t
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Set working directory (adjust the path accordingly)

In [ ]:
#os.chdir('/Users/fabianfuentes/Library/CloudStorage/OneDrive-InstitutoTecnologicoydeEstudiosSuperioresdeMonterrey/cursos/Econometrics/econometrics_mek/Week 2/')

Load data (using pandas instead of foreign library)

In [ ]:
jobcorpsfile = "https://raw.githubusercontent.com/emolinaperez/econometrics_mek/main/Week%202/data/jobcorps.dta"
JobCorps = pd.read_stata(jobcorpsfile)

Explore data file

In [ ]:
# Display the first 5 rows of the DataFrame
JobCorps.head()

In [ ]:
# Get the dimensions of the DataFrame
JobCorps.shape

In [ ]:
# Display attributes of the DataFrame
print("Columns:", JobCorps.columns.tolist())  # Column names
print("Index:", JobCorps.index)  # Row indices
print("Data Types:\n", JobCorps.dtypes)  # Data types of each column

In [ ]:
# Structure of the DataFrame
JobCorps.info()

In [ ]:
# Summary statistics for the DataFrame
print(JobCorps.describe())

Explore in class characteristics of differnet variables

Compare hispanics in control and treatment group

First compare means


In control group

In [ ]:
print(JobCorps[JobCorps['treatmnt'] == 0]['hispanic'].mean())

In [ ]:
print(JobCorps[JobCorps['treatmnt'] == 0]['hispanic'].sum())

In treatment group

In [ ]:
print(JobCorps[JobCorps['treatmnt'] == 1]['hispanic'].mean())

In [ ]:
print(JobCorps[JobCorps['treatmnt'] == 1]['hispanic'].sum())

Difference in means test (using statsmodels.stats.weightstats)

In [ ]:
# Calculate means and confidence intervals for "hispanic" by treatment group
treatment_means = JobCorps.groupby('treatmnt').apply(
    lambda group: pd.Series({
        'mean_hispanic': group['hispanic'].mean(skipna=True),
        'ci_lower': group['hispanic'].mean(skipna=True) - t.ppf(0.975, len(group) - 1) * group['hispanic'].std(skipna=True) / np.sqrt(len(group)),
        'ci_upper': group['hispanic'].mean(skipna=True) + t.ppf(0.975, len(group) - 1) * group['hispanic'].std(skipna=True) / np.sqrt(len(group))
    })
).reset_index()

treatment_means

In [ ]:
# Initialize a list to store the results
results = []

grouped = JobCorps.groupby('treatmnt')

# Iterate through each group to calculate statistics manually
for treatment_type, group in grouped:
    # Remove NaN values
    hispanic_values = group['hispanic'].dropna()

    # Sample size (n)
    n = len(hispanic_values)

    # Calculate mean
    mean_hispanic = hispanic_values.mean()

    # Calculate standard deviation
    std_dev = hispanic_values.std()

    # Calculate standard error
    std_error = std_dev / np.sqrt(n)

    # Degrees of freedom
    df = n - 1

    # Confidence level
    conf_level = 0.95
    t_value = t.ppf((1 + conf_level) / 2, df)  # t-value for 95% CI

    # Calculate confidence intervals
    ci_lower = mean_hispanic - t_value * std_error
    ci_upper = mean_hispanic + t_value * std_error

    # Append results as a dictionary
    results.append({
        'treatment_type': treatment_type,
        'mean_hispanic': mean_hispanic,
        'ci_lower_hispanic': ci_lower,
        'ci_upper_hispanic': ci_upper
    })

# Step 3: Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df

In [ ]:
# Bar plot with confidence intervals
plt.figure(figsize=(8, 6))

# Bar plot for mean_hispanic
sns.barplot(
    x='treatmnt',
    y='mean_hispanic',
    data=treatment_means,
    palette='viridis',
    alpha=0.6
)

# Add error bars for confidence intervals
plt.errorbar(
    x=treatment_means['treatmnt'],
    y=treatment_means['mean_hispanic'],
    yerr=[
        treatment_means['mean_hispanic'] - treatment_means['ci_lower'], # Lower error
        treatment_means['ci_upper'] - treatment_means['mean_hispanic']  # Upper error
    ],
    fmt='none',
    capsize=5,
    color='black'
)

# Labels and title
plt.title("Mean Proportion of Hispanics by Treatment", fontsize=14)
plt.xlabel("Treatment (0 = Control, 1 = Treated)", fontsize=12)
plt.ylabel("Mean Proportion of Hispanics", fontsize=12)
plt.xticks(ticks=[0, 1], labels=["Control (0)", "Treated (1)"], fontsize=10)
plt.legend(title="Treatment", labels=["Control", "Treated"], loc="upper left")

# Minimal theme
sns.despine()
plt.tight_layout()

plt.show();

In [ ]:
# Bar plot with confidence intervals using Seaborn
plt.figure(figsize=(8, 6))

sns.barplot(
    x="treatmnt",
    y="hispanic",
    data=JobCorps,
    ci=95,                # Add 95% confidence intervals
)

# Add labels and title
plt.title("Mean Proportion of Hispanics by Treatment", fontsize=14)
plt.xlabel("Treatment (0 = Control, 1 = Treated)", fontsize=12)
plt.ylabel("Mean Proportion of Hispanics", fontsize=12)
plt.xticks(ticks=[0, 1], labels=["Control (0)", "Treated (1)"], fontsize=10)
plt.legend(title="Treatment", loc="upper left")
plt.show();

In [ ]:
from scipy.stats import ttest_ind, t

# Separate the groups
group_0 = JobCorps[JobCorps['treatmnt'] == 0]['hispanic'].dropna()
group_1 = JobCorps[JobCorps['treatmnt'] == 1]['hispanic'].dropna()

# Perform the two-sample t-test (Welch's t-test for unequal variances)
t_stat, p_value = ttest_ind(group_0, group_1, equal_var=False)

# Calculate confidence interval for the difference in means
mean_group_0 = group_0.mean()
mean_group_1 = group_1.mean()
mean_diff = mean_group_0 - mean_group_1
std_error = np.sqrt(group_0.var(ddof=1) / len(group_0) + group_1.var(ddof=1) / len(group_1))
df = len(group_0) + len(group_1) - 2
t_crit = t.ppf(0.975, df)  # Two-tailed test at 95% confidence level
ci_lower = mean_diff - t_crit * std_error
ci_upper = mean_diff + t_crit * std_error

# Hypotheses
null_hypothesis = "The means of the two groups are equal (mean_group_0 = mean_group_1)."
alternative_hypothesis = "The means of the two groups are not equal (mean_group_0 ≠ mean_group_1)."

# Decision rule (alpha = 0.05)
alpha = 0.05
decision = "Reject the null hypothesis" if p_value < alpha else "Fail to reject the null hypothesis"

# Display the results
print("Two-Sample T-Test:")
print(f"Hypotheses:")
print(f"Null Hypothesis: {null_hypothesis}")
print(f"Alternative Hypothesis: {alternative_hypothesis}")
print(f"\nResults:")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")
print(f"Confidence Interval: [{ci_lower:.8f}, {ci_upper:.8f}]")
print(f"Mean in Group 0: {mean_group_0:.7f}")
print(f"Mean in Group 1: {mean_group_1:.7f}")
print(f"\nDecision: {decision}")

In [ ]:
# Separate the groups
group_0 = JobCorps[JobCorps['treatmnt'] == 0]['hispanic'].dropna()
group_1 = JobCorps[JobCorps['treatmnt'] == 1]['hispanic'].dropna()

# Perform the two-sample t-test (Welch's t-test for unequal variances)
t_stat, p_value = ttest_ind(group_0, group_1, equal_var=False)

# Calculate confidence interval for the difference in means
mean_group_0 = group_0.mean()
mean_group_1 = group_1.mean()
mean_diff = mean_group_0 - mean_group_1
std_error = np.sqrt(group_0.var(ddof=1) / len(group_0) + group_1.var(ddof=1) / len(group_1))
df = len(group_0) + len(group_1) - 2
t_crit = t.ppf(0.975, df)  # Two-tailed test at 95% confidence level
ci_lower = mean_diff - t_crit * std_error
ci_upper = mean_diff + t_crit * std_error

# Hypotheses
null_hypothesis = "The means of the two groups are equal (mean_group_0 = mean_group_1)."
alternative_hypothesis = "The means of the two groups are not equal (mean_group_0 ≠ mean_group_1)."

# Decision rule (alpha = 0.05)
alpha = 0.05
decision = "Reject the null hypothesis" if p_value < alpha else "Fail to reject the null hypothesis"

# Display the results
print("Two-Sample T-Test:")
print(f"Hypotheses:")
print(f"Null Hypothesis: {null_hypothesis}")
print(f"Alternative Hypothesis: {alternative_hypothesis}")
print(f"\nResults:")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")
print(f"Confidence Interval: [{ci_lower:.8f}, {ci_upper:.8f}]")
print(f"Mean in Group 0: {mean_group_0:.7f}")
print(f"Mean in Group 1: {mean_group_1:.7f}")
print(f"\nDecision: {decision}")

use OLS model to see of randomization was sucessful

 first define vector of variables of interest

In [ ]:
# Select column names from index 19 to 36 (Python uses 0-based indexing)
target_vars = JobCorps.columns[18:36].to_list()
target_vars

In [ ]:
# Exclude columns containing "earnq"
target_vars = [var for var in target_vars if var != 'treatmnt']
target_vars

In [ ]:
# Summary statistics for the selected columns
JobCorps[target_vars].describe()

In [ ]:
# Select specific columns and calculate the correlation matrix
cor_matrix = JobCorps[["white", "black", "hispanic", "treatmnt"]].corr()
cor_matrix

In [ ]:
# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(
    cor_matrix,
    annot=True,          # Display correlation coefficients on the heatmap
    fmt=".2f",           # Format for the annotations
    cmap="coolwarm",     # Color map for the heatmap
    cbar=True            # Display color bar
)

Run the regression, first a simple example with 3 observable caracteristics

In [ ]:
model1 = smf.ols('treatmnt ~ white + black + hispanic', data=JobCorps).fit()
print(model1.summary())

In [ ]:
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

# Residual Plots
fig, ax = plt.subplots(2, 2, figsize=(12, 10))

# Residuals vs Fitted
sns.residplot(x=model1.fittedvalues, y=model1.resid, lowess=True, line_kws={'color': 'red'}, ax=ax[0, 0])
ax[0, 0].axhline(0, color='gray', linestyle='--')
ax[0, 0].set_title("Residuals vs Fitted")
ax[0, 0].set_xlabel("Fitted Values")
ax[0, 0].set_ylabel("Residuals")

# Normal Q-Q
sm.qqplot(model1.resid, line="45", fit=True, ax=ax[0, 1])
ax[0, 1].set_title("Normal Q-Q")

# Scale-Location (Spread-Location)
abs_resid = abs(model1.resid)**0.5
sns.scatterplot(x=model1.fittedvalues, y=abs_resid, ax=ax[1, 0])
ax[1, 0].axhline(0, color='gray', linestyle='--')
ax[1, 0].set_title("Scale-Location")
ax[1, 0].set_xlabel("Fitted Values")
ax[1, 0].set_ylabel("Sqrt(|Residuals|)")

# Residuals vs Leverage
sm.graphics.influence_plot(model1, criterion="cooks", size=2, ax=ax[1, 1])
ax[1, 1].set_title("Residuals vs Leverage")

plt.tight_layout()
plt.show()

Now all variables

First save the model

In [ ]:
# Dynamically create the formula
dependent_var = "treatmnt"
formula_model2 = f"{dependent_var} ~ {' + '.join(target_vars)}"
formula_model2

Second run regression

In [ ]:
# Fit the model using the dynamically created formula
model2 = smf.ols(formula=formula_model2, data=JobCorps).fit()
print(model2.summary())

Selection on observables


In [ ]:
dependent_var = "attrit"
formula_model3 = f"{dependent_var} ~ {' + '.join(target_vars)}"
formula_model3

In [ ]:
model3 = smf.ols(formula=formula_model3, data=JobCorps).fit()
print(model3.summary())

In [ ]:
# Filter rows with complete cases for the specified columns
test = JobCorps.dropna(subset=target_vars)
test.shape

In [ ]:
cor_matrix = test[target_vars].corr()
cor_matrix

Average Treatment Effect

In [ ]:
# Define the formula
formula_model_ATE = "earnq16 ~ treatmnt"
formula_model_ATE

In [ ]:
model_ATE = smf.ols('earnq16 ~ treatmnt', data=JobCorps).fit()
print(model_ATE.summary())

Is this a really ATE ?

Effect on the Treated

In [ ]:
JobCorps.loc[JobCorps['treatmnt'] == 1, 'earnq16'].describe()

Average Effect of Treatment on the Treated (ToT)


In [ ]:
ToT = JobCorps[JobCorps['treatmnt'] == 1]['earnq16'].mean()
ToT

In [ ]:
NT = JobCorps[JobCorps['treatmnt'] == 0]['earnq16'].mean()
NT

In [ ]:
print(ToT - NT)

Compare with regression coefficient

In [ ]:
print(model_ATE.params)

Check for correlation between residuals and treatment

In [ ]:
print(model_ATE.resid.corr(JobCorps['treatmnt'].dropna()))

no correlation at all, what does this mean?